# Segmented Representations

One common representation in evolutionary algorithms (EA) is that of a "segmented representation."  That is, each individual is comprised of a sequence of segments, which are themselves fixed-length sequences, and are usually binary, but needn't be.  Each segment represents a salient feature, such as a rule in a Pitt Approach system, or a convolutional layer and its hyperparameters, as is the case for Multi-node Evolutionary Neural Networks for Deep Learning (MENNDL).

There are two broad categories for these systems: those that have a fixed number of such segments, as is the case currently for MENNDL, and a dynamic number of segments, as is the case for Pitt Approach classifiers.

In this notebook we look at LEAP support for segmented representations, starting with initializers and decoders, and then looking at the mutation pipeline operator.  We then plug all that into a simple EA example.


In [7]:
import sys
import random
import functools

from leap_ec.binary_rep.initializers import create_binary_sequence
from leap_ec.segmented_rep.initializers import create_segmented_sequence
from leap_ec.segmented_rep.decoders import SegmentedDecoder
from leap_ec.binary_rep.decoders import BinaryToIntDecoder
from leap_ec.segmented_rep.ops import apply_mutation
from leap_ec.binary_rep.ops import perform_mutate_bitflip

In [2]:
# Create a genome of four segments of five binary digits.
seg = create_segmented_sequence(4, create_binary_sequence(5))
print(seg)

[[0, 1, 1, 1, 1], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 0]]


In [3]:
# Now create five genomes of varying length by passing in a function for `length` that provides an
# integer drawn from a distribution.
seqs = [] # Save sequences for next step
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=5), create_binary_sequence(5))
    print(i, seq)
    seqs.append(seq)

0 [[1, 0, 1, 0, 1], [0, 0, 1, 1, 1], [1, 0, 0, 0, 1], [0, 0, 1, 0, 0]]
1 [[1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 0, 1]]
2 [[1, 1, 1, 1, 0]]
3 [[0, 1, 1, 1, 1]]
4 [[1, 1, 1, 0, 0], [1, 1, 0, 0, 1]]


Now let's see about decoding those segments.  The segmented representation relies on a secondary decoder that's applied to each segment.  In this case, we'll just use a simple binary to int decoder on the segments we created in the previous step.

In [5]:
# We want each segment to have two integers from the five bits.
decoder = SegmentedDecoder(BinaryToIntDecoder(2,3)) 

for i, seq in enumerate(seqs):
    vals = decoder.decode(seq)
    print(i, vals)

0 [[2, 5], [0, 7], [2, 1], [0, 4]]
1 [[3, 0], [0, 5], [1, 5]]
2 [[3, 6]]
3 [[1, 7]]
4 [[3, 4], [3, 1]]


In [15]:
# And now for mutation, which shows that, on average, a single value is changed in an example individual.  The
# takeaway here is that segmented mutation just uses a mutator from another representation and naively applies it.
from leap_ec.individual import Individual
original = Individual([[0,0],[1,1]])
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=perform_mutate_bitflip))
print('mutated:', mutated)

original: [[0, 0], [1, 1]]
mutated: [[1, 0], [1, 1]]
